# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv('../output_data/cities.csv', index_col = False)
city_weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Make 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["Lat", "Lng"]].astype(float)
humidity = city_weather_df["Humidity"].astype(float)

# Create Heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down Melissa's best weather conditions for vacationing in
# A max temperature lower than 75 degrees but higher than 55.
# Wind speed less than 10 and moody clouds ;)
best_temp = city_weather_df ['Max Temp'] < 75
best_temp = city_weather_df ['Max Temp'] > 55
best_wind = city_weather_df ['Wind Speed'] < 10
best_cloudiness = city_weather_df ['Cloudiness'] > 60
best_fog = city_weather_df ['Humidity'] > 60
my_weather = best_temp & best_wind & best_cloudiness & best_fog

# Drop the rows with nulls
weirdo_vacations = city_weather_df[my_weather]
weirdo_vacations = weirdo_vacations.dropna()
weirdo_vacations = weirdo_vacations.reset_index()
weirdo_vacations.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
1,23,23,maragogi,97,BR,1558378610,69,-9.01,-35.22,83.94,9.42
2,45,45,rosita,72,NI,1558378762,71,13.92,-84.40,84.12,4.41
3,48,48,port alfred,67,ZA,1558378763,69,-33.59,26.89,61.00,8.01
4,91,91,bella vista,62,AR,1558378771,80,-28.51,-59.05,75.66,7.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel df and add a column for hotel name
hotel_df = pd.DataFrame(weirdo_vacations, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Hotel Name'])
hotel_df.head()

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,avarua,CK,-21.21,-159.78,71.60,NaN
1,maragogi,BR,-9.01,-35.22,83.94,NaN
2,rosita,NI,13.92,-84.40,84.12,NaN
3,port alfred,ZA,-33.59,26.89,61.00,NaN
4,bella vista,AR,-28.51,-59.05,75.66,NaN


In [7]:
counter = 0

for index, row in hotel_df.iterrows():

    # Set parameters to search for hotels with 5000 meters.
    params = {"location": f"{row['Lat']}, {row['Lng']}", "radius": 5000, "types": "lodging",
              "keyword": "Hotel", "key": g_key}
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a json response with the parameters above
    response = requests.get(base_url, params=params)
    
    # convert to json and print them easy to read
    response_j = response.json()
    print(json.dumps(response_j, indent=4, sort_keys=True))
    
    results = response_j['results']
    
    # Name the hotel if you get a positive result
    if len(results) > 0:
        name = response_j['results'][0]['name']
        hotel_df.iloc[counter, -1] = name
    counter = counter + 1

hotel_weirdo_df = hotel_df
hotel_weirdo_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.25519,
                    "lng": -159.72975
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.25376987010728,
                        "lng": -159.7282333701073
                    },
                    "southwest": {
                        "lat": -21.25646952989272,
                        "lng": -159.7309330298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Muri Beach Club Hotel Rarotonga",
            "photos": [
                {
                    "height": 2184,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/1070437177611626902

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKY0vwa-bQxCA0udZzaR8ZjKN1UBuahzfCvWByQpiEUEZUtLhYMuXX97LzIZWv9BF7t1tQnwqotStPVFCsLbRDj3xIffRHbaLtmewZRGdZk5qtIVuZiNE1mmtEsefUYXaiC-2AOyhxVH24fUYx42k1ujLFaZdaEo5IeZzx_SDxFKC7_kXk1PdOiCvM6wU248kL82UD7A9g4L5rQ4X9pWuTr49W4LMZSpkTzfILzwh75Zb_djpNVaABBP1d20Pn7qoxBCgelK3r_gxCdc477FNjxhI2prluVXdParIDFCWEsL3BgVDr9qFkc2rJCOesRMQYpIw5ugyxYfJjW_hEE9COBdo7aVIrfIlGX5qhkMDuS4WLntMqWJgfhpoSH3TRgikCS5SWLnNWsUrvCekySiFugOhtUUYpE_giM6CjWyUPCT2pyRlClQfcVEnWuL8Rorg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -9.007587899999999,
                    "lng": -35.2169647
                },
                "viewport": {
                    "northeast": {
                        "lat": -9.006226420107279,
                        "lng": -35.21563177010728
                    },
                    "southwest": {
                        "lat":

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 13.9240661,
                    "lng": -84.3969408
                },
                "viewport": {
                    "northeast": {
                        "lat": 13.92536857989272,
                        "lng": -84.39563717010728
                    },
                    "southwest": {
                        "lat": 13.92266892010728,
                        "lng": -84.39833682989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Terciopelo",
            "photos": [
                {
                    "height": 3456,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/118161517274619893459\">Marlon Mal

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.5904314,
                    "lng": 26.8858279
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.58916472010728,
                        "lng": 26.88700332989272
                    },
                    "southwest": {
                        "lat": -33.59186437989273,
                        "lng": 26.88430367010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "MyPond Hotel",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/111925452985229144220\">Rippen</a>"
  

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "next_page_token": "ATtYBwLcHy8aLl6rIVHE7mr31SrNclqJYkvCxVCs6kunNZmBhs0AicZJ5CtmDnkpkb92dRH50mWQEwu4LHOGgTgkVmY2Uf9J7Ab0-feX3SqglEHVRJeWkiMOx0pT2TDf9dLqQD9IUlwb9ZhX1a02i_CoCd3TgZO6b5uTY_rulrVYp00eqJtEKsxFnAOUtTNYAhGOAqFbEpiaNweQ5QdlMEqMrEzdcqwIFVu4ThGHDLBe6GcrWGM9_WBFse9vnXQW2xuYfaPdTDk3AF34YSjNADlLVfQZl_H9e7NO4N5MYogF73ZroTrNVLCYx0abcP5qyr9rCbQqjxxswgMs5wSiuTlVqWsFlzolPLdh1v6PBZoBdru8yde0Xvjsy4Pdxc3yaVPk38GTHvl5D9ITRWD4rC93YdwtFuIHwHJgBZKTMhI-fuQG3WKOePQIHdvp7mztLWq5AA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -0.8651291,
                    "lng": 131.2458257
                },
                "viewport": {
                    "northeast": {
                        "lat": -0.8

...

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 2.0344447,
                    "lng": 45.3033553
                },
                "viewport": {
                    "northeast": {
                        "lat": 2.035890029892722,
                        "lng": 45.30452867989273
                    },
                    "southwest": {
                        "lat": 2.033190370107278,
                        "lng": 45.30182902010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Jowhara International Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2448,
                    "html_attributions": [
                    

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 54.2363749,
                    "lng": -125.771098
                },
                "viewport": {
                    "northeast": {
                        "lat": 54.23759042989272,
                        "lng": -125.7700104201073
                    },
                    "southwest": {
                        "lat": 54.23489077010728,
                        "lng": -125.7727100798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Key-oh Lodge",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 3872,
                    "html_attributions": [
                        "<a hre

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKvKTVX8btDr6SqaTbIdbTRy_Frj4LLBA3WlFiaOi83cp1pNuPqmpfLZzA6gOQ0Qy281D4ofhZTgwsRvbndwE2L4s1FnlpfWiXvunlL6fhvCV4Hrs0oLKcSKCRlmDzPO4CWVehsd9BWivu-utR3S1xbo4UPTOVjL_Wq1IgcT4bpYSi0OHjG2MeQv0leXxwLMKbYA8Go7rYEd8ODi1_I70mbxrzzWMK4SrsievbSISmC3KY6QwylwQwIdCpQAaX5H6ekNJv0JlIl3k-KMDUh6jSs_m7XkG6EVmJK_ycFljgd7LeZ74t0b4ij_YHfA3ydkZK3vGDGTNwlRlRt_6S6mDSiy5fgI8vYEYycaujZWBj-M9drWJKLj4FCogVzX95Z9sGt4iXl0W8EYXTywJyKH10vZi8XrZL96DG5ME47R95NoHA82vXyp4-dqGo2FAIMWQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -0.936967,
                    "lng": 100.3776346
                },
                "viewport": {
                    "northeast": {
                        "lat": -0.9356404701072777,
                        "lng": 100.3788954298927
                    },
                    "southwest": {
                        "lat": -0.93834

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIoUyk1_uC7diSg1o1VMrjEF_dSx5YZ39i0e7ko0aeSKs5W7cOSQYettd3W5WJqEwYGNonlmFMI_j5-01MP4LZVW-tv5XYV5GheXdoTFOT7LAImZ4MfF6qKXZFCjHm-sXc6ZP-v2SKREWKzyHjcvHr6IEGfCmyqKnyuVoRcelyqH5UOdY9WMDOMJ5SyQaOfAloi1Id5R1h6jo-MXggF3miWCXgbapXCL2yOqLGDt06fIxugd0RJUUpoCKlctkKssk3zxcgoVL1m_kyjlu7Y87S1usqI7oyb_rvtE7kZ8fp46UrkcmF05EbNtmhFsmaU3DwlWPmXczboe1K1IAmz84nJlaEF47u7A0fFWMbRX9YPB2kWKWMdjQpzTzsZQ-mrWi-Nhzq7SoQLlXAQrTqDsdXlLTxdw8TFeglOWRgkHqDJjHckJiWeGGhVMzcBCpJW0A",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -0.8848802,
                    "lng": 119.8681502
                },
                "viewport": {
                    "northeast": {
                        "lat": -0.8836010201072778,
                        "lng": 119.8694591298927
                    },
                    "southwest": {
                        "lat": -0.8863

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,avarua,CK,-21.21,-159.78,71.60,Muri Beach Club Hotel Rarotonga
1,maragogi,BR,-9.01,-35.22,83.94,POUSADA DOS CABANOS
2,rosita,NI,13.92,-84.40,84.12,Hotel Terciopelo
3,port alfred,ZA,-33.59,26.89,61.00,MyPond Hotel
4,bella vista,AR,-28.51,-59.05,75.66,NaN
...,...,...,...,...,...,...
82,malindi,KE,-3.22,40.12,77.00,Driftwood Beach Club Ltd
83,mogadishu,SO,2.04,45.34,80.88,Jowhara International Hotel
84,burns lake,CA,54.23,-125.76,56.40,Key-oh Lodge
85,padang,ID,-0.92,100.36,81.24,Polamas Residence


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_weirdo_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Plot marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(marker_layer)